In [18]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [19]:
%autoreload 1
%aimport offnet

In [ ]:
import os.path

In [2]:
import torch

In [3]:
import torchvision

In [4]:
import visdom

In [5]:
import matplotlib.pyplot as plt
import numpy as np

In [6]:
import ignite
import ignite.metrics

In [6]:
import importlib

In [7]:
transform = torchvision.transforms.Compose(
    (
#         transforms.Pad(padding=2, fill=0),
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    )
)


In [8]:
train_dataset = torchvision.datasets.FashionMNIST(
    os.path.expanduser("~/.pytorchdataset/fashion-mnist"),
    train=True,
    transform=transform,
    download=True
)

In [9]:
test_dataset = torchvision.datasets.FashionMNIST(
    os.path.expanduser("~/.pytorchdataset/fashion-mnist"),
    train=False,
    transform=transform,
    download=True
)

In [10]:
train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=32,
    shuffle=True,
    num_workers=2
)

In [11]:
test_loader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=32,
    shuffle=False,
    num_workers=2
)

In [16]:
model = offnet.Net()

In [21]:
loss = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [22]:
trainer = ignite.engine.create_supervised_trainer(model, optimizer, loss)

In [23]:
evaluator = ignite.engine.create_supervised_evaluator(
    model,
    metrics={
        'accuracy': ignite.metrics.CategoricalAccuracy(),
        'loss': ignite.metrics.Loss(loss)
    }
)

In [24]:
def log_training_results(trainer):
    evaluator.run(train_loader)
    metrics = evaluator.state.metrics
    print("Training Results - Epoch: {}  Avg accuracy: {:.2f} Avg loss: {:.2f}"
          .format(trainer.state.epoch, metrics['accuracy'], metrics['loss']))

trainer.add_event_handler(ignite.engine.Events.EPOCH_COMPLETED, log_training_results)

In [25]:
def log_validation_results(trainer):
    evaluator.run(test_loader)
    metrics = evaluator.state.metrics
    print("Validation Results - Epoch: {}  Avg accuracy: {:.2f} Avg loss: {:.2f}"
          .format(trainer.state.epoch, metrics['accuracy'], metrics['nll']))

trainer.add_event_handler(ignite.engine.Events.EPOCH_COMPLETED, log_validation_results)

In [26]:
trainer.add_event_handler(
    ignite.engine.Events.EPOCH_COMPLETED,
    ignite.handlers.ModelCheckpoint('/tmp/models', 'offnet', save_interval=2, n_saved=2, create_dir=True, save_as_state_dict=True),
    {
        'model': model,
        'optimizer': optimizer,
    }
)

In [29]:
trainer.run(train_loader, max_epochs=100)

Process Process-6:
Process Process-5:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/Users/twer/workspace/offnet.pytorch/envs/default/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/Users/twer/workspace/offnet.pytorch/envs/default/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/Users/twer/workspace/offnet.pytorch/envs/default/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/twer/workspace/offnet.pytorch/envs/default/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/twer/workspace/offnet.pytorch/envs/default/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 52, in _worker_loop
    r = index_queue.get()
  File "/Users/twer/workspace/offnet.pytorch/envs/default/lib/python3.6/site-packages/torch/utils/data/dataloader.py

KeyboardInterrupt: 

In [30]:
evaluation_state = evaluator.run(test_loader)

evaluation_state.metrics

{'accuracy': 0.6872, 'loss': 0.8093685500144958}

In [31]:
print(evaluation_state.metrics)

{'accuracy': 0.6872, 'loss': 0.8093685500144958}
